#**DATA GATHERING**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
df_raw = pd.read_csv("Corona.csv")

In [3]:
df_raw.head()

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
0,1,11-03-2020,TRUE,FALSE,TRUE,FALSE,FALSE,negative,None,None,Abroad
1,2,11-03-2020,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
2,3,11-03-2020,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
3,4,11-03-2020,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Abroad
4,5,11-03-2020,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Contact with confirmed


In [4]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278848 entries, 0 to 278847
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   Ind_ID               278848 non-null  int64 
 1   Test_date            278848 non-null  object
 2   Cough_symptoms       278848 non-null  object
 3   Fever                278848 non-null  object
 4   Sore_throat          278848 non-null  object
 5   Shortness_of_breath  278848 non-null  object
 6   Headache             278848 non-null  object
 7   Corona               278848 non-null  object
 8   Age_60_above         278848 non-null  object
 9   Sex                  278848 non-null  object
 10  Known_contact        278848 non-null  object
dtypes: int64(1), object(10)
memory usage: 23.4+ MB


In [5]:
df_raw["Test_date"] = pd.to_datetime(df_raw["Test_date"], format='%d-%m-%Y')

In [6]:
df_new_dataset = df_raw[(df_raw["Test_date"])>"15-04-2020"]

In [7]:
df_new_dataset.head()

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
163189,163190,2020-04-16,True,False,False,False,False,negative,None,female,Abroad
163190,163191,2020-04-16,False,False,False,False,False,negative,None,female,Other
163191,163192,2020-04-16,True,False,False,False,False,negative,None,female,Abroad
163192,163193,2020-04-16,False,False,False,False,False,negative,None,male,Other
163193,163194,2020-04-16,False,False,False,False,False,negative,None,male,Other


In [8]:
for i in df_new_dataset.columns[2:]:
  print(df_new_dataset[i].value_counts())

False    102542
True      13117
Name: Cough_symptoms, dtype: int64
False    108968
True       6691
Name: Fever, dtype: int64
False    115636
True         23
Name: Sore_throat, dtype: int64
False    115641
True         18
Name: Shortness_of_breath, dtype: int64
False    115633
True         26
Name: Headache, dtype: int64
negative    111541
positive      2698
other         1420
Name: Corona, dtype: int64
None    115659
Name: Age_60_above, dtype: int64
female    58226
male      55550
None       1883
Name: Sex, dtype: int64
Other                     105570
Abroad                      8082
Contact with confirmed      2007
Name: Known_contact, dtype: int64


In [9]:
df_new_dataset["Corona"] = df_new_dataset["Corona"].replace("other",np.nan)

In [10]:
df_new_dataset.dropna(inplace = True)

In [11]:
df = df_raw[df_raw["Test_date"]<="15-04-2020"]

In [12]:
df.head()

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
0,1,2020-03-11,TRUE,FALSE,TRUE,FALSE,FALSE,negative,None,None,Abroad
1,2,2020-03-11,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
2,3,2020-03-11,FALSE,TRUE,FALSE,FALSE,FALSE,positive,None,None,Abroad
3,4,2020-03-11,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Abroad
4,5,2020-03-11,TRUE,FALSE,FALSE,FALSE,FALSE,negative,None,None,Contact with confirmed


In [13]:
df.shape

(163189, 11)

#**DATA INFORMATION**

In [14]:
df.nunique()

Ind_ID                 163189
Test_date                  36
Cough_symptoms              5
Fever                       5
Sore_throat                 5
Shortness_of_breath         5
Headache                    5
Corona                      3
Age_60_above                3
Sex                         3
Known_contact               3
dtype: int64

In [15]:
for i in df.columns[2:]:
  print(df[i].value_counts())

FALSE    108837
False     24989
TRUE      21983
True       7128
None        252
Name: Cough_symptoms, dtype: int64
FALSE    119070
False     28806
TRUE      11750
True       3311
None        252
Name: Fever, dtype: int64
False    96948
FALSE    64337
TRUE      1198
True       705
None         1
Name: Sore_throat, dtype: int64
False    97201
FALSE    64428
TRUE      1107
True       452
None         1
Name: Shortness_of_breath, dtype: int64
False    96693
FALSE    64107
TRUE      1428
True       960
None         1
Name: Headache, dtype: int64
negative    148686
positive     12031
other         2472
Name: Corona, dtype: int64
No      125703
Yes      25825
None     11661
Name: Age_60_above, dtype: int64
male      73577
female    71932
None      17680
Name: Sex, dtype: int64
Other                     137171
Abroad                     17386
Contact with confirmed      8632
Name: Known_contact, dtype: int64


In [16]:
df.isnull().sum()

Ind_ID                 0
Test_date              0
Cough_symptoms         0
Fever                  0
Sore_throat            0
Shortness_of_breath    0
Headache               0
Corona                 0
Age_60_above           0
Sex                    0
Known_contact          0
dtype: int64

In [17]:
for i in df.columns[2:]:
  print("\nUnique values in '{}' are :\n".format(i),df[i].unique())


Unique values in 'Cough_symptoms' are :
 ['TRUE' 'FALSE' 'None' False True]

Unique values in 'Fever' are :
 ['FALSE' 'TRUE' 'None' False True]

Unique values in 'Sore_throat' are :
 ['TRUE' 'FALSE' 'None' False True]

Unique values in 'Shortness_of_breath' are :
 ['FALSE' 'TRUE' 'None' False True]

Unique values in 'Headache' are :
 ['FALSE' 'TRUE' 'None' False True]

Unique values in 'Corona' are :
 ['negative' 'positive' 'other']

Unique values in 'Age_60_above' are :
 ['None' 'No' 'Yes']

Unique values in 'Sex' are :
 ['None' 'male' 'female']

Unique values in 'Known_contact' are :
 ['Abroad' 'Contact with confirmed' 'Other']


In [18]:
df.shape

(163189, 11)

#**FEATURE ENGINEERING**

In [19]:
symptoms = ["Cough_symptoms", "Fever", "Sore_throat", "Shortness_of_breath", "Headache", "Sex", "Corona"]
for i in symptoms:
  df[i] = df[i].replace("TRUE",1)
  df[i] = df[i].replace("FALSE",0)
  df[i] = df[i].replace(True,1)
  df[i] = df[i].replace(False,0)
  df[i] = df[i].replace("None",np.nan)
  df[i] = df[i].replace("Yes",1)
  df[i] = df[i].replace("No",0)
  df[i] = df[i].replace("other",np.nan)
  # df[i] = df[i].replace("positive",1)
  # df[i] = df[i].replace("negative",0)

In [20]:
df.isnull().sum()

Ind_ID                     0
Test_date                  0
Cough_symptoms           252
Fever                    252
Sore_throat                1
Shortness_of_breath        1
Headache                   1
Corona                  2472
Age_60_above               0
Sex                    17680
Known_contact              0
dtype: int64

In [21]:
df.head()

,Ind_ID,Test_date,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Age_60_above,Sex,Known_contact
0,1,2020-03-11,1.0,0.0,1.0,0.0,0.0,negative,None,NaN,Abroad
1,2,2020-03-11,0.0,1.0,0.0,0.0,0.0,positive,None,NaN,Abroad
2,3,2020-03-11,0.0,1.0,0.0,0.0,0.0,positive,None,NaN,Abroad
3,4,2020-03-11,1.0,0.0,0.0,0.0,0.0,negative,None,NaN,Abroad
4,5,2020-03-11,1.0,0.0,0.0,0.0,0.0,negative,None,NaN,Contact with confirmed


In [22]:
df.shape

(163189, 11)

In [23]:
df.dropna(inplace = True)

In [24]:
df.shape

(143305, 11)

In [25]:
df.drop(["Ind_ID", "Test_date", "Age_60_above"], axis=1, inplace =True)

#**FEATURE SCALING**

In [26]:
df.head()

,Cough_symptoms,Fever,Sore_throat,Shortness_of_breath,Headache,Corona,Sex,Known_contact
13727,1.0,0.0,0.0,0.0,0.0,negative,male,Abroad
13728,1.0,0.0,0.0,0.0,0.0,negative,female,Abroad
13730,1.0,1.0,0.0,0.0,0.0,positive,female,Other
13734,0.0,0.0,0.0,0.0,0.0,negative,male,Other
13735,0.0,0.0,0.0,0.0,0.0,negative,female,Other


In [27]:
X = df.drop(["Corona"], axis=1)
y = df["Corona"]

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [29]:
X_train.shape, y_train.shape

((114644, 7), (114644,))

In [30]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
ct = ColumnTransformer(transformers=[
        ('cat1', OneHotEncoder(sparse = False, drop = "first"), ["Sex", "Known_contact"])
                                    ], remainder='passthrough')


#Using fit_transform to the train data.
X_train = ct.fit_transform(X_train)

#"transform" only to avoid data leakage.
X_test = ct.transform(X_test)

In [31]:
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [32]:
X_train.shape, y_train.shape

((114644, 8), (114644,))

In [33]:
X_test.shape, y_test.shape

((28661, 8), (28661,))

In [34]:
#Importing Standardization from sklearn
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

#Using fit_transform to the train data.
X_train = scaler.fit_transform(X_train)

#"transform" only to avoid data leakage.
X_test = scaler.transform(X_test)

#**MODEL BUILDING**

###Logistic

In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)
y_pred = log_reg.predict(X_test)
print("Accuracy of Logistic Regression Model : {}%".format(round(accuracy_score(y_pred, y_test)*100)))
print("Classification Report of Logistic Regression Model : \n",classification_report(y_pred, y_test))

Accuracy of Logistic Regression Model : 95%
Classification Report of Logistic Regression Model : 
               precision    recall  f1-score   support

           0       0.99      0.96      0.98     27177
           1       0.54      0.78      0.64      1484

    accuracy                           0.95     28661
   macro avg       0.76      0.87      0.81     28661
weighted avg       0.96      0.95      0.96     28661



In [36]:
confusion_matrix(y_pred, y_test)

array([[26174,  1003],
       [  323,  1161]])

In [37]:
from sklearn.model_selection import GridSearchCV
parameter = {"penalty":["l1", "l2", "elasticnet"],
             "C" : [1,2,3,4,5,6,10,20,40,50],
             "max_iter" : [100,200,300,400,500]}
Classfier_Reg = GridSearchCV(log_reg,param_grid=parameter, scoring="accuracy", cv=10)

In [38]:
Classfier_Reg.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 10, 20, 40, 50],
                         'max_iter': [100, 200, 300, 400, 500],
                         'penalty': ['l1', 'l2', 'elasticnet']},
             scoring='accuracy')

In [39]:
print(Classfier_Reg.best_params_)
print(Classfier_Reg.best_score_)
y_pred_tune_grid = Classfier_Reg.predict(X_test)
print("Accuracy is {}%".format(round(accuracy_score(y_pred_tune_grid, y_test)*100)))
print(classification_report(y_pred_tune_grid, y_test))

{'C': 4, 'max_iter': 100, 'penalty': 'l2'}
0.9523045509422315
Accuracy is 95%
              precision    recall  f1-score   support

           0       0.99      0.96      0.98     27177
           1       0.54      0.78      0.64      1484

    accuracy                           0.95     28661
   macro avg       0.76      0.87      0.81     28661
weighted avg       0.96      0.95      0.96     28661



In [40]:
confusion_matrix(y_pred_tune_grid, y_test)

array([[26174,  1003],
       [  323,  1161]])

###Decision Tree

In [41]:
from sklearn.tree import DecisionTreeClassifier
dec_tree = DecisionTreeClassifier()
dec_tree.fit(X_train,y_train)
y_pred_dec = dec_tree.predict(X_test)
accuracy_score(y_pred_dec, y_test)*100

95.99804612539688

In [42]:
confusion_matrix(y_pred_dec, y_test)

array([[26261,   911],
       [  236,  1253]])

In [43]:
print(classification_report(y_pred_dec, y_test))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     27172
           1       0.58      0.84      0.69      1489

    accuracy                           0.96     28661
   macro avg       0.79      0.90      0.83     28661
weighted avg       0.97      0.96      0.96     28661



###KNN

In [44]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)
y_pred_knn = knn.predict(X_test)
accuracy_score(y_pred_knn, y_test)*100

95.79916960329368

In [45]:
confusion_matrix(y_pred_knn, y_test)

array([[26126,   833],
       [  371,  1331]])

In [46]:
print(classification_report(y_pred_knn, y_test))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     26959
           1       0.62      0.78      0.69      1702

    accuracy                           0.96     28661
   macro avg       0.80      0.88      0.83     28661
weighted avg       0.96      0.96      0.96     28661



###Random Forest

In [47]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier()
random_forest.fit(X_train,y_train)
y_pred_rfc = random_forest.predict(X_test)
accuracy_score(y_pred_rfc, y_test)*100

96.00851331077074

In [48]:
confusion_matrix(y_pred_rfc, y_test)

array([[26261,   908],
       [  236,  1256]])

In [49]:
print(classification_report(y_pred_rfc, y_test))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     27169
           1       0.58      0.84      0.69      1492

    accuracy                           0.96     28661
   macro avg       0.79      0.90      0.83     28661
weighted avg       0.97      0.96      0.96     28661



###Gradient Boost

In [50]:
from sklearn.ensemble import GradientBoostingClassifier
grad_boost = GradientBoostingClassifier()
grad_boost.fit(X_train,y_train)
y_pred_gbc = grad_boost.predict(X_test)
accuracy_score(y_pred_gbc, y_test)*100

96.01200237256202

In [51]:
confusion_matrix(y_pred_gbc, y_test)

array([[26261,   907],
       [  236,  1257]])

In [52]:
print(classification_report(y_pred_gbc, y_test))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     27168
           1       0.58      0.84      0.69      1493

    accuracy                           0.96     28661
   macro avg       0.79      0.90      0.83     28661
weighted avg       0.97      0.96      0.96     28661



###SVC

In [53]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train,y_train)
y_pred_svm = svm.predict(X_test)
accuracy_score(y_pred_svm, y_test)*100

96.01200237256202

In [54]:
confusion_matrix(y_pred_svm, y_test)

array([[26261,   907],
       [  236,  1257]])

In [55]:
print(classification_report(y_pred_svm, y_test))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98     27168
           1       0.58      0.84      0.69      1493

    accuracy                           0.96     28661
   macro avg       0.79      0.90      0.83     28661
weighted avg       0.97      0.96      0.96     28661

